In [23]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from itertools import combinations
import numpy as np
from pandas import Index
import seaborn as sns
from sklearn.feature_selection import SelectKBest
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Loading the dataset

In [24]:
sheet_id = "11wZvIszOgW-be-Vzs8FTop7NaJn8pNJQeqfcxr2Kh70"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

table = pd.read_csv(url)
table

,Size,Atom,G_h (eV),rcov,rd,Ef,IE,EN,eps_d,n,q (e),d_occ,d_unocc
0,Periodic Graphene,Sc,-0.190,1.70,0.310,-8.282,6.561,1.36,1.064,3,1.640,0.85,4.17
1,Periodic Graphene,Ti,-0.500,1.60,0.280,-8.326,6.827,1.54,0.998,4,1.404,0.83,3.58
2,Periodic Graphene,V,-0.270,1.53,0.260,-7.494,6.746,1.63,1.316,5,1.341,1.22,2.57
3,Periodic Graphene,Cr,0.310,1.39,0.250,-6.604,6.766,1.66,0.944,6,1.263,2.36,0.98
4,Periodic Graphene,Mn,0.430,1.39,0.230,-6.520,7.434,1.55,-0.513,7,1.279,1.33,2.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Small nanographene,Ag,2.048,1.45,0.385,-3.331,7.576,1.93,-4.020,11,0.773,0.03,0.45
95,Small nanographene,Cd,0.245,1.44,0.370,-2.291,8.994,1.69,-2.120,12,1.069,0.43,0.01
96,Small nanographene,Hf,-1.127,1.75,0.630,-9.987,6.824,1.30,0.453,4,1.571,0.54,2.16
97,Small nanographene,Ta,-1.092,1.70,0.605,-9.573,7.887,1.50,0.120,5,1.605,1.01,1.94


Separating descriptors/feature and output

In [25]:
X = table.iloc[:,3:]
y = table.iloc[:,2]
columnnames = X.columns
columnnames = list(columnnames)
#columnnames

Expanding feature set by using mathematical operators like *, /, ^k 

In [26]:
for i in range(0,X.shape[1]):
  currcol = columnnames[i]
  inv = columnnames[i] + '^-1'
  sq = columnnames[i] + '^2'
  cube = columnnames[i] + '^3'

  columnnames.append(inv)
  columnnames.append(sq)
  columnnames.append(cube)

  table[inv] = table[currcol].apply(lambda x: x**-1)
  table[sq] = table[currcol].apply(lambda x: x**2)
  table[cube] = table[currcol].apply(lambda x: x**3)
  for j in range(i+1, X.shape[1]):
    secondcol = columnnames[j]
    mul = columnnames[i] + " x " + columnnames[j]
    div1 = columnnames[i] + " / " + columnnames[j]
    div2 = columnnames[j] + " / " + columnnames[i]
    columnnames.append(mul)
    columnnames.append(div1)
    columnnames.append(div2)
    table[mul] = round( table.apply(lambda row: row[currcol] * row[secondcol], axis = 1) , 4 )
    table[div1] = round( table.apply(lambda row: row[currcol] / row[secondcol], axis = 1) , 4 )
    table[div2] = round( table.apply(lambda row: row[secondcol] / row[currcol], axis = 1) , 4 )



In [27]:
columnnames = table.columns.tolist()

# Feature Selection

**Process:**


1.   Find out Strong feature
> A strong feature is one which has high correlation with the output ( G(h) )


2.  From the given strong features, we find out which features have low correlation with each other, i.e which features are **independent**

Creating correlation matrix between all features and output

In [28]:
corr_matrix = table.corr()
corr_matrix

,G_h (eV),rcov,rd,Ef,IE,EN,eps_d,n,q (e),d_occ,...,d_unocc / q (e),d_occ^-1,d_occ^2,d_occ^3,d_occ x d_unocc,d_occ / d_unocc,d_unocc / d_occ,d_unocc^-1,d_unocc^2,d_unocc^3
G_h (eV),1.000000,-0.735092,-0.561784,0.486171,0.434896,0.302545,-0.734205,0.744729,-0.699383,-0.033747,...,-0.406603,0.350382,0.024468,0.047130,-0.318937,0.357888,0.240015,0.215325,-0.400022,-0.287771
rcov,-0.735092,1.000000,0.701220,-0.383215,-0.541954,-0.429506,0.631862,-0.803848,0.794285,-0.427623,...,0.331217,-0.054407,-0.454150,-0.440599,-0.057900,-0.420044,0.031653,-0.316690,0.485443,0.408142
rd,-0.561784,0.701220,1.000000,-0.257692,-0.008695,0.166198,0.321226,-0.376324,0.368481,-0.175544,...,-0.017007,0.118156,-0.207131,-0.229948,-0.142490,-0.245218,0.147620,-0.263822,0.032384,-0.034941
Ef,0.486171,-0.383215,-0.257692,1.000000,0.471568,0.182006,-0.746154,0.697502,-0.329669,-0.441928,...,-0.371431,0.418836,-0.350473,-0.297899,-0.472749,0.333181,0.324133,0.513806,-0.312991,-0.244455
IE,0.434896,-0.541954,-0.008695,0.471568,1.000000,0.493123,-0.676807,0.763135,-0.520784,0.040474,...,-0.441517,0.248237,0.073005,0.074848,-0.260254,0.542037,0.180918,0.519929,-0.395369,-0.333793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_occ / d_unocc,0.357888,-0.420044,-0.245218,0.333181,0.542037,0.084521,-0.490309,0.504132,-0.327335,0.043268,...,-0.559903,-0.113418,0.056842,0.067793,-0.343522,1.000000,-0.163174,0.831748,-0.331668,-0.234992
d_unocc / d_occ,0.240015,0.031653,0.147620,0.324133,0.180918,0.181753,-0.237760,0.129344,-0.027177,-0.337739,...,0.049684,0.961956,-0.237938,-0.189090,-0.250233,-0.163174,1.000000,-0.094863,0.008378,0.016118
d_unocc^-1,0.215325,-0.316690,-0.263822,0.513806,0.519929,-0.119219,-0.522032,0.473328,-0.099090,-0.239357,...,-0.466745,-0.033223,-0.196331,-0.157947,-0.344529,0.831748,-0.094863,1.000000,-0.237245,-0.162242
d_unocc^2,-0.400022,0.485443,0.032384,-0.312991,-0.395369,-0.399977,0.461843,-0.595162,0.508935,-0.182915,...,0.858439,-0.123600,-0.238440,-0.236103,0.393553,-0.331668,0.008378,-0.237245,1.000000,0.968485


Number of strong features to be selected

In [29]:
k = 50

**Finding out strong features**


> * First convert correlation matrix into positive values, since positive/negative correlation is not of importance here.
* Sort the features based on correlation coefficient in descending order
*The top 'k' values in this list are the required 'strong features'

strong_features contains the correlation value of strong variables 

strong_feature_names is a list containing names of all the strong features

In [30]:
abs_corr_mat = corr_matrix.abs()
strong_features = abs_corr_mat.loc['G_h (eV)','rcov':]
strong_features = strong_features.sort_values(ascending=False)[:k]
strong_feature_names = strong_features.index.tolist()[:k]
print(strong_features)
strong_feature_names

eps_d / q (e)    0.780046
n / rcov         0.777110
EN x eps_d       0.770779
n / q (e)        0.765635
Ef x eps_d       0.761262
n / IE           0.759433
n^2              0.747123
n                0.744729
rcov^-1          0.736829
rcov             0.735092
rd x q (e)       0.734889
eps_d            0.734205
rd / n           0.732607
eps_d / IE       0.732122
eps_d / rcov     0.732048
rcov^2           0.730064
rcov x q (e)     0.729794
n^3              0.729452
rcov x eps_d     0.729059
IE x eps_d       0.723753
rcov^3           0.722640
Ef x q (e)       0.715576
eps_d x n        0.713012
rd x eps_d       0.707833
EN x n           0.706147
EN / n           0.706119
IE x n           0.699456
q (e)            0.699383
rd x d_unocc     0.699202
q (e)^2          0.696646
IE / n           0.694601
n / rd           0.691703
eps_d / n        0.691419
eps_d / EN       0.686106
q (e) / IE       0.682216
q (e)^3          0.680361
rcov / n         0.672868
IE / q (e)       0.672483
q (e) / n   

['eps_d / q (e)',
 'n / rcov',
 'EN x eps_d',
 'n / q (e)',
 'Ef x eps_d',
 'n / IE',
 'n^2',
 'n',
 'rcov^-1',
 'rcov',
 'rd x q (e)',
 'eps_d',
 'rd / n',
 'eps_d / IE',
 'eps_d / rcov',
 'rcov^2',
 'rcov x q (e)',
 'n^3',
 'rcov x eps_d',
 'IE x eps_d',
 'rcov^3',
 'Ef x q (e)',
 'eps_d x n',
 'rd x eps_d',
 'EN x n',
 'EN / n',
 'IE x n',
 'q (e)',
 'rd x d_unocc',
 'q (e)^2',
 'IE / n',
 'n / rd',
 'eps_d / n',
 'eps_d / EN',
 'q (e) / IE',
 'q (e)^3',
 'rcov / n',
 'IE / q (e)',
 'q (e) / n',
 'rd x Ef',
 'eps_d x q (e)',
 'rcov x n',
 'rd / IE',
 'n^-1',
 'IE / rcov',
 'Ef / n',
 'rcov / IE',
 'q (e)^-1',
 'EN / rd',
 'n / EN']



---



Now that we have found the strong features, we will create a correlation matrix consisting of **only** the strong features

In [31]:
strong_feature_corr = pd.DataFrame()
df_entry = []
for feature in strong_feature_names:
  strong_feature_corr[feature] = abs_corr_mat.loc[strong_feature_names, feature]
strong_feature_corr

,eps_d / q (e),n / rcov,EN x eps_d,n / q (e),Ef x eps_d,n / IE,n^2,n,rcov^-1,rcov,...,eps_d x q (e),rcov x n,rd / IE,n^-1,IE / rcov,Ef / n,rcov / IE,q (e)^-1,EN / rd,n / EN
eps_d / q (e),1.000000,0.839018,0.985880,0.790666,0.855461,0.829173,0.884506,0.860778,0.582113,0.581060,...,0.922217,0.859889,0.418285,0.741459,0.693561,0.777748,0.674630,0.584284,0.404874,0.777975
n / rcov,0.839018,1.000000,0.846109,0.900044,0.876374,0.948744,0.975309,0.987234,0.870304,0.869630,...,0.872345,0.928922,0.646290,0.933471,0.914399,0.910023,0.915899,0.721233,0.646335,0.890598
EN x eps_d,0.985880,0.846109,1.000000,0.767969,0.861727,0.812347,0.886593,0.864376,0.598093,0.597220,...,0.944915,0.858168,0.414297,0.750443,0.731890,0.786492,0.706925,0.550637,0.402197,0.765154
n / q (e),0.790666,0.900044,0.767969,1.000000,0.807761,0.911198,0.891037,0.917383,0.730350,0.747405,...,0.709010,0.899071,0.471485,0.876247,0.772083,0.818432,0.802418,0.936424,0.466290,0.708922
Ef x eps_d,0.855461,0.876374,0.861727,0.807761,1.000000,0.826290,0.835761,0.860204,0.767712,0.766852,...,0.887127,0.802997,0.515178,0.838846,0.820050,0.778548,0.827106,0.660550,0.567830,0.712999
n / IE,0.829173,0.948744,0.812347,0.911198,0.826290,1.000000,0.936785,0.958965,0.797753,0.813056,...,0.814432,0.930266,0.591678,0.929581,0.754374,0.922449,0.796635,0.761287,0.606997,0.843757
n^2,0.884506,0.975309,0.886593,0.891037,0.835761,0.936785,1.000000,0.989481,0.748157,0.750672,...,0.889690,0.970328,0.554139,0.891167,0.858090,0.900209,0.850131,0.681956,0.523132,0.912663
n,0.860778,0.987234,0.864376,0.917383,0.860204,0.958965,0.989481,1.000000,0.794098,0.803848,...,0.875165,0.975840,0.560784,0.945333,0.880593,0.935175,0.893021,0.733157,0.541708,0.883066
rcov^-1,0.582113,0.870304,0.598093,0.730350,0.767712,0.797753,0.748157,0.794098,1.000000,0.993939,...,0.674605,0.650222,0.831421,0.821104,0.861260,0.757796,0.877560,0.640133,0.857786,0.715112
rcov,0.581060,0.869630,0.597220,0.747405,0.766852,0.813056,0.750672,0.803848,0.993939,1.000000,...,0.672124,0.671186,0.826742,0.846961,0.855019,0.790138,0.886848,0.667552,0.834248,0.698502


Now, we find out which features are least correlated to each other by the 



In [32]:
corr_pairs = strong_feature_corr.where(np.triu(np.ones(strong_feature_corr.shape), k=1).astype(bool))
corr_pairs = corr_pairs.unstack().dropna().sort_values()

corr_pair_list = []
for i in range(corr_pairs.shape[0]):
  corr_pair_list.append(corr_pairs[i])

print(corr_pairs)
corr_pair_list

EN / rd       rd x eps_d      0.270175
rd x Ef       IE / q (e)      0.312629
q (e)^-1      rd x Ef         0.323645
rd / IE       rd x eps_d      0.349960
EN / rd       IE / q (e)      0.352930
                                ...   
eps_d / rcov  eps_d           0.995621
IE x eps_d    eps_d / rcov    0.996540
rcov x eps_d  eps_d / IE      0.996603
rcov^2        rcov            0.998586
rcov^3        rcov^2          0.998680
Length: 1225, dtype: float64


[0.2701750236839557,
 0.3126294524466192,
 0.3236447852906171,
 0.3499601660795997,
 0.3529303720635524,
 0.37170051123265074,
 0.37603907266581743,
 0.3815382517017313,
 0.386141029963823,
 0.3940774856457816,
 0.39641128565500083,
 0.40219712386104617,
 0.40487355750056747,
 0.4070864888578832,
 0.41215845683444646,
 0.4142965772780857,
 0.4182852699022528,
 0.42439788829814284,
 0.43075377982208274,
 0.43331203993445827,
 0.433971573164432,
 0.44226927330253996,
 0.44527846497046975,
 0.4549139248908596,
 0.4551984979808552,
 0.4560967550153829,
 0.4569572604950275,
 0.45718042777700296,
 0.46060552020673395,
 0.4607283294295154,
 0.4638757889902982,
 0.46465789300076826,
 0.46616637098631974,
 0.4662904423055545,
 0.4669323367804661,
 0.4683534170252991,
 0.4687778709755307,
 0.4691700069496198,
 0.46965007249272145,
 0.4706533072303461,
 0.47148520499324814,
 0.47482249726442677,
 0.4782854231927354,
 0.48055496907098155,
 0.48303552206954486,
 0.48511704681546014,
 0.486378547655

In [33]:
corr_analysis = pd.DataFrame()
corr_analysis['Feature 1'] = corr_pairs.index.get_level_values(0).tolist()
corr_analysis['Feature 2'] = corr_pairs.index.get_level_values(1).tolist()
corr_analysis['Correlation Coefficient'] = corr_pair_list
corr_analysis

,Feature 1,Feature 2,Correlation Coefficient
0,EN / rd,rd x eps_d,0.270175
1,rd x Ef,IE / q (e),0.312629
2,q (e)^-1,rd x Ef,0.323645
3,rd / IE,rd x eps_d,0.349960
4,EN / rd,IE / q (e),0.352930
...,...,...,...
1220,eps_d / rcov,eps_d,0.995621
1221,IE x eps_d,eps_d / rcov,0.996540
1222,rcov x eps_d,eps_d / IE,0.996603
1223,rcov^2,rcov,0.998586


In [34]:
str_feature_importance = pd.DataFrame(columns = ['Feature', 'Index Sum'])
str_feature_importance['Feature'] = strong_feature_names

for index, feature in enumerate(strong_feature_names):
  indexes = corr_analysis.index[corr_analysis['Feature 1'] == feature].tolist()
  indexes.extend(corr_analysis.index[corr_analysis['Feature 2'] == feature].tolist())
  index_sum = sum(indexes)
  str_feature_importance.iat[index, 1] = index_sum
  indexes = []
  
str_feature_importance = str_feature_importance.sort_values(by = 'Index Sum', ignore_index=True)
#str_feature_importance.reset_index()
str_feature_importance

,Feature,Index Sum
0,EN / rd,10791
1,rd x Ef,11911
2,rd / IE,12119
3,rd x d_unocc,15824
4,q (e)^-1,18771
5,n / rd,19141
6,rd x q (e),19952
7,IE / q (e),24073
8,eps_d x n,25898
9,rd x eps_d,26731


In [35]:
selected_features = table.loc[:,str_feature_importance.loc[:,'Feature']]
selected_features['G_h (eV)'] = table.loc[:,'G_h (eV)']
#selected_features = table.loc[:,strong_feature_names]
#selected_features.insert(0,'G_h (eV)',table.iloc[:,2],True)
selected_features

,EN / rd,rd x Ef,rd / IE,rd x d_unocc,q (e)^-1,n / rd,rd x q (e),IE / q (e),eps_d x n,rd x eps_d,...,q (e) / n,IE / n,rcov / n,n^-1,IE x n,n / IE,n^2,n,n / rcov,"(G_h (eV), 0)"
0,4.3871,-2.5674,0.0472,1.2927,0.609756,9.6774,0.5084,4.0006,3.192,0.3298,...,0.5467,2.1870,0.5667,0.333333,19.683,0.4572,9,3,1.7647,-0.190
1,5.5000,-2.3313,0.0410,1.0024,0.712251,14.2857,0.3931,4.8625,3.992,0.2794,...,0.3510,1.7068,0.4000,0.250000,27.308,0.5859,16,4,2.5000,-0.500
2,6.2692,-1.9484,0.0385,0.6682,0.745712,19.2308,0.3487,5.0306,6.580,0.3422,...,0.2682,1.3492,0.3060,0.200000,33.730,0.7412,25,5,3.2680,-0.270
3,6.6400,-1.6510,0.0369,0.2450,0.791766,24.0000,0.3157,5.3571,5.664,0.2360,...,0.2105,1.1277,0.2317,0.166667,40.596,0.8868,36,6,4.3165,0.310
4,6.7391,-1.4996,0.0309,0.5658,0.781861,30.4348,0.2942,5.8124,-3.591,-0.1180,...,0.1827,1.0620,0.1986,0.142857,52.038,0.9416,49,7,5.0360,0.430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,5.0130,-1.2824,0.0508,0.1733,1.293661,28.5714,0.2976,9.8008,-44.220,-1.5477,...,0.0703,0.6887,0.1318,0.090909,83.336,1.4520,121,11,7.5862,2.048
95,4.5676,-0.8477,0.0411,0.0037,0.935454,32.4324,0.3955,8.4135,-25.440,-0.7844,...,0.0891,0.7495,0.1200,0.083333,107.928,1.3342,144,12,8.3333,0.245
96,2.0635,-6.2918,0.0923,1.3608,0.636537,6.3492,0.9897,4.3437,1.812,0.2854,...,0.3928,1.7060,0.4375,0.250000,27.296,0.5862,16,4,2.2857,-1.127
97,2.4793,-5.7917,0.0767,1.1737,0.623053,8.2645,0.9710,4.9140,0.600,0.0726,...,0.3210,1.5774,0.3400,0.200000,39.435,0.6340,25,5,2.9412,-1.092


In [36]:
y = selected_features.iloc[:,0]
X = selected_features.iloc[:,1:]

In [37]:
def Randtraintest(X, y, test_size):
    '''
    This function splits the descriptors (X) and output (y) points into train
    and test sets. The size of test set depends on the number of folds of CV.
    '''

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

testtraindata = Randtraintest(X, y, 0.3)
X_test = testtraindata[1]
y_test = testtraindata[3]
y_train = testtraindata[2]
X_test_idx = X_test.index.values

def lasso_regression_grid_search(X, y):
    # Create a pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LassoCV(cv=5))
    ])
    
    pipeline.fit(X, y)

    # Get the coefficients of the best estimator in the original scale
    scaler = pipeline.named_steps['scaler']
    regressor = pipeline.named_steps['regressor']
    coef = regressor.coef_ / scaler.scale_
    
    return pipeline, coef
    

def rmse(y_train, y_test, trainpred, testpred):
    
    # compares the predicted points to the actual points and returns a RMSE val
    train_rmse = mean_squared_error(y_train, trainpred, squared=False)
    test_rmse = mean_squared_error(y_test, testpred, squared=False)

    return train_rmse, test_rmse

pipeline, coef = lasso_regression_grid_search(testtraindata[0], testtraindata[2])
trainpred = pipeline.predict(testtraindata[0])
testpred = pipeline.predict(testtraindata[1])
trainrmse, testrmse = rmse(y_train, y_test, trainpred, testpred)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.095e-01, tolerance: 4.330e-02
  model = cd_fast.enet_coordinate_descent(


In [38]:
coef_table = pd.DataFrame(columns= selected_features.columns[1:].tolist())
coef_table = coef_table.append(pd.Series(coef,index=coef_table.columns), ignore_index=True)
print("train RMSE = ", trainrmse, " test RMSE = ", testrmse)
coef_table

train RMSE =  0.09085725296886156  test RMSE =  0.18160365484493438


,rd x Ef,rd / IE,rd x d_unocc,q (e)^-1,n / rd,rd x q (e),IE / q (e),eps_d x n,rd x eps_d,q (e),...,q (e) / n,IE / n,rcov / n,n^-1,IE x n,n / IE,n^2,n,n / rcov,G_h (eV)
0,0.143778,-35.341556,0.143659,0.0,0.146605,0.0,-0.0,0.004277,-0.405892,-0.487177,...,-0.0,-0.399061,-0.0,-0.0,-0.0,1.062927,-0.0,0.0,0.0,-0.0
